In [1]:
import mechanicalsoup
from bs4 import BeautifulSoup
import re
import MySQLdb


In [2]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://onlinetrading.nse.co.ke/tradeweb111/")

<Response [200]>

In [ ]:
file = open('pass.txt', 'r') 
password =str(file.read())

In [3]:
browser.select_form(nr=0)

# User credentials
browser['txtLogin'] = 'karis'
browser['txtPassword'] = password

# Login
browser.submit_selected()



<Response [200]>

In [4]:
browser.open('https://onlinetrading.nse.co.ke/tradeweb111/MarketDetails.aspx?wl=Default')
#https://onlinetrading.nse.co.ke/tradeweb111/MarketDetails.aspx?wl=Default
page = browser.get_current_page()

In [5]:
soup = BeautifulSoup(str(page), 'lxml')
script = soup.find_all("script")


In [6]:
jss =  str(script[22])
str_replace_qoutes = jss.replace('\"', '\\"')
str_minus_semi_colon = str_replace_qoutes.replace(";"," ")
str_minus_sdarr = str_minus_semi_colon.replace("sDArr","")
str_minus_equal = str_minus_sdarr.replace("=","")

#everything before sdarr
head, sep, tail = str_minus_equal.partition('(jsiScripsInMW,50) ')
#everything after sdarr
head2, sep2, tail2 = tail.partition('  jsiMWrefreshDelay')
result = re.sub("[\(\[].*?[\)\]]", "", head2)
arr = result.split("' '")
arr[0] = arr[0].replace("\'","")
arr[-1] = arr[-1].replace("\'","")

In [7]:
sdarr = []
sec = []
count =1
for i in range(len(arr)):
    count = count + 1
    sec.append(arr[i])
    if count ==22:
        count = 1
        sdarr.append(sec)
        sec =[]


In [8]:
#print(sdarr)

In [9]:
def sortlist(mylist):
    newlist=[]
    for item in mylist:
        if float(item) > 0:
            newlist.append(item)
    
    if newlist == []:
        newlist = [str(0.00), str(0.00)]
    return newlist

In [10]:
sql = "INSERT INTO market_details (security, last_price, demand_qty, demand_price, supply_price, supply_qty, last_qty, high, low) VALUES "
for row in sdarr:
    
    sub_row = [row[2], row[8], row[9], row[10], row[11], row[12]]
    high_low = sortlist(sub_row)
    sql = sql + "( '"+row[0]+"', "+ row[2]+", "+row[5]+", "+row[6]+", "+row[8]+", "+row[7]+", "+row[4]+", "+max(high_low)+", "+min(high_low)+"),"
sql = sql[:-1] + ";"    
db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                 user="root",         # your username
                 passwd="",  # your password
                 db="nse")        # name of the data base

db.query(sql)
db.commit()

db.close()


In [11]:
print(len(sdarr))

70
